# Table of Contents <a id='back'></a>

* [Introduction](#intro)
* [Open the data file and have a look at the general information.](#1.-info)
* [Data exploration](#1.-explore)
* [Data transformation](#transform)
* [Working with missing values](#missing)
* [Restoring missing values in `total_income`](#missing-income)
* [Restoring values in `days_employed`](#missing-days)
* [Categorization of data](#categorize)
* [Checking the Hypotheses](#hypo)
* [General Conclusion ](#end)

# Analyzing borrowers’ risk of defaulting <a id='intro'></a>

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.

## Open the data file and have a look at the general information. <a id='info'></a>

In [122]:
# Loading all the libraries
import pandas as pd

In [123]:
# Load the data
credit_score = pd.read_csv('/datasets/credit_scoring_eng.csv')

credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Task 1. Data exploration <a id='explore'></a>

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [124]:
# Let's see how many rows and columns our dataset has

num_rows, num_columns = credit_score.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)

Number of rows: 21525
Number of columns: 12


In [125]:
# let's print the first N rows

N = 5  

credit_score.head(N)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [126]:
# Get info on data
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [127]:
# Let's look in the filtered table at the the first column with missing data

# Filtering the credit score to select only columns with missing values
filtered_table = credit_score.loc[:, credit_score.isnull().any(axis=0)]

# Getting the first column with missing data
first_column_with_missing_data = filtered_table.columns[0]

print("First column with missing data:", first_column_with_missing_data)

First column with missing data: days_employed


In [128]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.


# Count the number of missing values in each row
missing_values_count = credit_score.isnull().sum(axis=1)

# Get the unique counts and their frequencies
count_distribution = missing_values_count.value_counts()

print("Distribution of missing values counts in rows:")
print(count_distribution)


Distribution of missing values counts in rows:
0    19351
2     2174
dtype: int64



<b>Based on the distribution of missing values counts in rows: </b>

<ol>
<li>The majority of rows (19,351 out of the total) do not have any missing values.</li>
<li>There are 2,174 rows that contain two missing values.</li>
<li>From the data given, the distribution suggests that the missing values are not symmetric or evenly distributed across rows.</li>
<li>The majority of rows have no missing values, while a smaller portion of rows might have two missing values.</li>
<li>Early conclusion suggest an imbalance in the occurrence of missing values within the dataset.</li>
</ol>

**Intermediate conclusion**

1. Missing data could be due to the specific client characteristic
1. Check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic

In [129]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

# Check for missing values in each column
missing_values_per_column = credit_score.isnull().sum()

print("Missing values per column:")
print(missing_values_per_column)

Missing values per column:
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [130]:
# Step 1: Verify if the number of rows in the filtered table matches the number of missing values
missing_values_count = credit_score.isnull().sum(axis=1)
count_distribution = missing_values_count.value_counts()
print("Distribution of missing values counts in rows:")
print(count_distribution)

Distribution of missing values counts in rows:
0    19351
2     2174
dtype: int64


In [131]:
# Step 2: Calculate the percentage of missing values compared to the whole dataset
total_missing_values = credit_score.isnull().sum().sum()
total_entries = credit_score.size
percentage_missing = (total_missing_values / total_entries) * 100
rounded_percentage_missing = round(percentage_missing, 2)
print("Percentage of missing values in the whole dataset:", rounded_percentage_missing)

Percentage of missing values in the whole dataset: 1.68


In [132]:
# Create a new DataFrame copy with rows containing missing values in 'days_employed' or 'total_income'
df_na = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)].copy()

# Printing the new DataFrame containing rows with missing values
print("DataFrame containing rows with missing values:")
df_na

DataFrame containing rows with missing values:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [133]:
# Checking distribution
# Checking the distribution of numerical columns using .describe()
numerical_columns = ['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income']

numerical_stats = credit_score[numerical_columns].describe()

print("Distribution of Numerical Columns:")
numerical_stats

Distribution of Numerical Columns:


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [134]:
df_na['income_type'].value_counts(normalize = True)

employee         0.508280
business         0.233671
retiree          0.189972
civil servant    0.067617
entrepreneur     0.000460
Name: income_type, dtype: float64

In [135]:
credit_score['income_type'].value_counts(normalize = True)

employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
entrepreneur                   0.000093
unemployed                     0.000093
paternity / maternity leave    0.000046
student                        0.000046
Name: income_type, dtype: float64

In [136]:
df_na['education'].value_counts(normalize = True)

secondary education    0.647654
bachelor's degree      0.228151
SECONDARY EDUCATION    0.030819
Secondary Education    0.029899
some college           0.025299
Bachelor's Degree      0.011500
BACHELOR'S DEGREE      0.010580
primary education      0.008740
SOME COLLEGE           0.003220
Some College           0.003220
PRIMARY EDUCATION      0.000460
Primary Education      0.000460
Name: education, dtype: float64

In [137]:
credit_score['education'].value_counts(normalize = True)

secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
Graduate Degree        0.000046
GRADUATE DEGREE        0.000046
Name: education, dtype: float64

<b>Primary finding are:<b>

Most of the loaner was employee with secondary education was the main core in the loaner education


**Possible reasons for missing values in data**

In [138]:
# Checking the distribution in the whole dataset
# Checking the distribution of numerical columns using .describe()
numerical_columns = ['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'debt', 'total_income']

numerical_stats = credit_score[numerical_columns].describe()

print("Distribution of Numerical Columns:")
numerical_stats

Distribution of Numerical Columns:


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Intermediate conclusion**

In conclusion, the dataset contains some data quality issues such as missing values, negative values in 'days_employed' and 'children', and unusual ages. These anomalies need to be addressed through data cleaning and preprocessing steps before conducting any further analysis or drawing meaningful conclusions from the data. Additionally, it is essential to investigate the categorical columns ('education_id' and 'family_status_id') to understand their labels and meanings.

In [139]:
# Check for other reasons and patterns that could lead to missing values


# Step 1: Investigate the distribution of missing values based on 'income_type'
missing_data_income_type = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)]
income_type_distribution = missing_data_income_type['income_type'].value_counts()

print("Distribution of Missing Values Based on 'income_type':")
print(income_type_distribution)

# Step 2: Check for patterns in missing values based on other columns (e.g., 'children' and 'family_status')
# Filter for rows with missing values in 'days_employed' and 'total_income'
missing_data_pattern = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)]

# Check the distribution of missing values based on 'children' and 'family_status'
pattern_distribution = missing_data_pattern.groupby(['children', 'family_status'])['family_status'].count()

print("\nDistribution of Missing Values Based on 'children' and 'family_status':")
pattern_distribution


Distribution of Missing Values Based on 'income_type':
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Distribution of Missing Values Based on 'children' and 'family_status':


children  family_status    
-1        married                2
          unmarried              1
 0        civil partnership    288
          divorced              71
          married              759
          unmarried            231
          widow / widower       90
 1        civil partnership    101
          divorced              32
          married              299
          unmarried             42
          widow / widower        1
 2        civil partnership     39
          divorced               7
          married              146
          unmarried              9
          widow / widower        3
 3        civil partnership     11
          divorced               1
          married               23
          unmarried              1
 4        civil partnership      1
          married                5
          unmarried              1
 5        civil partnership      1
 20       civil partnership      1
          divorced               1
          married          

**Intermediate conclusion**

The accidental data input may play a role here.

In [140]:
# Checking for other patterns - explain which


# Step 1: Investigate the distribution of missing values based on 'income_type'
missing_data_income_type = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)]
income_type_distribution = missing_data_income_type['income_type'].value_counts()

print("Distribution of Missing Values Based on 'income_type':")
print(income_type_distribution)

# Step 2: Investigate the distribution of missing values based on 'education'
missing_data_education = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)]
education_distribution = missing_data_education['education'].value_counts()

print("\nDistribution of Missing Values Based on 'education':")
print(education_distribution)

# Step 3: Investigate the distribution of missing values based on 'purpose'
missing_data_purpose = credit_score[credit_score[['days_employed', 'total_income']].isnull().any(axis=1)]
purpose_distribution = missing_data_purpose['purpose'].value_counts()

print("\nDistribution of Missing Values Based on 'purpose':")
print(purpose_distribution)


Distribution of Missing Values Based on 'income_type':
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Distribution of Missing Values Based on 'education':
secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
SOME COLLEGE              7
Some College              7
PRIMARY EDUCATION         1
Primary Education         1
Name: education, dtype: int64

Distribution of Missing Values Based on 'purpose':
having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72
transactions with my real estate            71
pu

**Conclusions**

Income Type: 
>The majority of missing values for 'income_type' fall into the category of "employee" (1105 missing values), followed by "business" (508 missing values) and there is only one missing value for "entrepreneur." This indicates that the dataset mainly consists of employees and that missing values are more common among employees and business owners.

Education: 
>The highest number of missing values for 'education' is associated with "secondary education" (1408 missing values), which is the most common category. It seems that missing values are more prevalent for individuals with lower educational backgrounds.

Purpose: 
>The distribution of missing values based on 'purpose' is more scattered, with several different purposes having a significant number of missing values. The highest number of missing values is related to the purpose of "having a wedding" (92 missing values), followed by various real estate and housing-related purposes. This indicates that there may be incomplete information for specific purposes within the dataset.

In summary, it appears that the missing values in this dataset are not uniformly distributed across different categories. The prevalence of missing values is influenced by the specific attributes, such as income type, education level, and loan purpose.  Proper imputation techniques or statistical methods should be employed to address missing values and minimize their impact on the analysis results.

## Data transformation <a id='transform'></a>

In [141]:
# Let's see all values in education column to check if and what spellings will need to be fixed

# Check all unique values in the 'education' column
unique_education_values = credit_score['education'].unique()

print("Unique values in the 'education' column:")
print(unique_education_values)


Unique values in the 'education' column:
["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [142]:
# Fix the registers if required
# Convert 'education' column values to lowercase for standardization
credit_score['education'] = credit_score['education'].str.lower()

# Check all unique values in the 'education' column after standardization
unique_education_values = credit_score['education'].unique()

print("Unique values in the 'education' column after standardization:")
print(unique_education_values)

Unique values in the 'education' column after standardization:
["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [143]:
# Checking all the values in the column to make sure we fixed them

# Check all unique values in the 'education' column
unique_education_values = credit_score['education'].unique()

print("Unique values in the 'education' column:")
print(unique_education_values)

Unique values in the 'education' column:
["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


In [144]:
# Let's see the distribution of values in the `children` column

# Count the frequency of each value in the 'children' column
children_distribution = credit_score['children'].value_counts()

# Sort the values by index (number of children) for better visualization
children_distribution = children_distribution.sort_index()


In [145]:
print("Distribution of values in the 'children' column:")
children_distribution 

Distribution of values in the 'children' column:


-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

There is an error regarding the number of children. The (-1) and 20 child is illogical.

In [146]:
# Count the frequency of each value in the 'children' column
children_distribution = credit_score['children'].value_counts()

# Sort the values by index (number of children) for better visualization
children_distribution = children_distribution.sort_index()

# Calculate the percentage of clients for each category
total_clients = len(credit_score)
children_percentage = (children_distribution / total_clients) * 100

print("Percentage distribution of values in the 'children' column:")
print(children_percentage)

Percentage distribution of values in the 'children' column:
-1      0.218351
 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 4      0.190476
 5      0.041812
 20     0.353078
Name: children, dtype: float64


In [147]:
# [fix the data based on your decision, assuming the typo error present in the data]

# Fix the data by replacing -1 with 1 and 20 with 2 in the 'children' column
credit_score['children'] = credit_score['children'].replace(-1, 1)
credit_score['children'] = credit_score['children'].replace(20, 2)

# Count the frequency of each value in the 'children' column after fixing the data
children_distribution = credit_score['children'].value_counts()

# Sort the values by index (number of children) for better visualization
children_distribution = children_distribution.sort_index()

# Calculate the percentage of clients for each category
total_clients = len(credit_score)
children_percentage = (children_distribution / total_clients) * 100

print("Percentage distribution of values in the 'children' column after fixing the data:")
print(children_percentage)


Percentage distribution of values in the 'children' column after fixing the data:
0    65.732869
1    22.601626
2     9.900116
3     1.533101
4     0.190476
5     0.041812
Name: children, dtype: float64


In [148]:
# Checking the `children` column again to make sure it's all fixed

print("Distribution of values in the 'children' column after fixing the data:")
print(children_distribution)

Distribution of values in the 'children' column after fixing the data:
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [149]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage

# Check for negative values in the 'days_employed' column
negative_days_employed = credit_score[credit_score['days_employed'] < 0]

# Check for extremely large positive values in the 'days_employed' column (assuming 50 years of experience is an outlier)
extremely_large_days_employed = credit_score[credit_score['days_employed'] > 18250]

# Calculate the total number of rows with problematic data
total_problematic_rows = len(negative_days_employed) + len(extremely_large_days_employed)

# Calculate the percentage of problematic data compared to the whole dataset
total_rows = len(credit_score)
percentage_problematic_data = (total_problematic_rows / total_rows) * 100

print("Percentage of problematic data in the 'days_employed' column:", percentage_problematic_data)


Percentage of problematic data in the 'days_employed' column: 89.90011614401858


In [150]:
# Address the problematic values, if they exist

# Address negative values by taking the absolute value
credit_score['days_employed'] = credit_score['days_employed'].abs()

# Address extremely large positive values (considering 50 years as an outlier)
max_employment_days = 18250
credit_score.loc[credit_score['days_employed'] > max_employment_days, 'days_employed'] = None

# Check the updated distribution of 'days_employed' after addressing problematic values
days_employed_stats = credit_score['days_employed'].describe()


In [151]:
# Check the result - make sure it's fixed

print("Updated Distribution of 'days_employed' after addressing problematic values:")
print(days_employed_stats)

Updated Distribution of 'days_employed' after addressing problematic values:
count    15905.000000
mean      2352.007700
std       2300.804988
min         24.141633
25%        756.281915
50%       1629.997862
75%       3156.957393
max      17615.563266
Name: days_employed, dtype: float64


In [152]:
# Check the `dob_years` for suspicious values and count the percentage

# Check for suspicious values in the 'dob_years' column (age cannot be zero or negative)
suspicious_dob_years = credit_score[credit_score['dob_years'] <= 0]

# Calculate the total number of rows with suspicious values
total_suspicious_rows = len(suspicious_dob_years)

# Calculate the percentage of suspicious values compared to the whole dataset
total_rows = len(credit_score)
percentage_suspicious_data = (total_suspicious_rows / total_rows) * 100

print("Percentage of suspicious values in the 'dob_years' column:", percentage_suspicious_data)


Percentage of suspicious values in the 'dob_years' column: 0.4692218350754936


In [153]:
# Address the issues in the `dob_years` column, if they exist

# Replace suspicious values (ages <= 0) with NaN to mark them as missing values
credit_score['dob_years'] = credit_score['dob_years'].apply(lambda age: age if age > 0 else None)

# Impute missing ages with the median age
median_age = credit_score['dob_years'].median()
credit_score['dob_years'].fillna(median_age, inplace=True)

# Check the updated distribution of 'dob_years' after addressing the issues
dob_years_stats = credit_score['dob_years'].describe()

print("Updated Distribution of 'dob_years' after addressing the issues:")
print(dob_years_stats)



Updated Distribution of 'dob_years' after addressing the issues:
count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


In [154]:
# Check the result - make sure it's fixed

print("Updated Distribution of 'dob_years' after addressing the issues:")
print(dob_years_stats)


Updated Distribution of 'dob_years' after addressing the issues:
count    21525.000000
mean        43.495145
std         12.218213
min         19.000000
25%         34.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


In [155]:
# Let's see the values for the column

# Check the unique values in the 'family_status' column
family_status_unique_values = credit_score['family_status'].unique()

print("Unique values in the 'family_status' column:")
print(family_status_unique_values)


Unique values in the 'family_status' column:
['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


In [156]:
# Address the problematic values in `family_status`, if they exist

# Convert all values to lowercase in the 'family_status' column
credit_score['family_status'] = credit_score['family_status'].str.lower()

# Capitalize the first letter of each family status
credit_score['family_status'] = credit_score['family_status'].str.capitalize()

# Check the updated unique values in the 'family_status' column
family_status_unique_values = credit_score['family_status'].unique()

print("Updated unique values in the 'family_status' column:")
print(family_status_unique_values)



Updated unique values in the 'family_status' column:
['Married' 'Civil partnership' 'Widow / widower' 'Divorced' 'Unmarried']


In [157]:
# Check the result - make sure it's fixed
print("Updated unique values in the 'family_status' column:")
print(family_status_unique_values)

Updated unique values in the 'family_status' column:
['Married' 'Civil partnership' 'Widow / widower' 'Divorced' 'Unmarried']


In [158]:
# Let's see the values in the column

# Check the unique values in the 'gender' column
gender_unique_values = credit_score['gender'].unique()

print("Unique values in the 'gender' column:")
print(gender_unique_values)

Unique values in the 'gender' column:
['F' 'M' 'XNA']


In [159]:
gender_xna_count = credit_score['gender'].value_counts().get('XNA', 0)
print("Number of occurrences of 'XNA' in the 'gender' column:", gender_xna_count)

Number of occurrences of 'XNA' in the 'gender' column: 1


In [160]:
# Address the problematic values, if they exist
# Convert all values to lowercase in the 'gender' column
credit_score['gender'] = credit_score['gender'].str.lower()

# Capitalize the first letter of each gender
credit_score['gender'] = credit_score['gender'].str.capitalize()

# Check the updated unique values in the 'gender' column
gender_unique_values = credit_score['gender'].unique()

print("Updated unique values in the 'gender' column:")
print(gender_unique_values)


Updated unique values in the 'gender' column:
['F' 'M' 'Xna']


In [161]:
# Check the result - make sure it's fixed

print("Updated unique values in the 'gender' column:")
print(gender_unique_values)


Updated unique values in the 'gender' column:
['F' 'M' 'Xna']


In [162]:
# Drop rows with 'Xna' value in the 'gender' column
credit_score = credit_score[credit_score['gender'] != 'Xna']

# Check the updated unique values in the 'gender' column
gender_unique_values = credit_score['gender'].unique()

print("Updated unique values in the 'gender' column:")
print(gender_unique_values)

Updated unique values in the 'gender' column:
['F' 'M']


In [163]:
gender_xna_count = credit_score['gender'].value_counts().get('XNA', 0)
print("Number of occurrences of 'XNA' in the 'gender' column:", gender_xna_count)

Number of occurrences of 'XNA' in the 'gender' column: 0


In [164]:
# Let's see the values in the column
# Check the unique values in the 'income_type' column
income_type_unique_values = credit_score['income_type'].unique()

print("Unique values in the 'income_type' column:")
print(income_type_unique_values)


Unique values in the 'income_type' column:
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']


In [165]:
# Address the problematic values, if they exist

# Convert all values to lowercase in the 'income_type' column
credit_score['income_type'] = credit_score['income_type'].str.lower()

# Capitalize the first letter of each income type
credit_score['income_type'] = credit_score['income_type'].str.capitalize()

# Check the updated unique values in the 'income_type' column
income_type_unique_values = credit_score['income_type'].unique()

print("Updated unique values in the 'income_type' column:")
print(income_type_unique_values)


Updated unique values in the 'income_type' column:
['Employee' 'Retiree' 'Business' 'Civil servant' 'Unemployed'
 'Entrepreneur' 'Student' 'Paternity / maternity leave']


In [166]:
# Check the result - make sure it's fixed
print("Updated unique values in the 'income_type' column:")
print(income_type_unique_values)

Updated unique values in the 'income_type' column:
['Employee' 'Retiree' 'Business' 'Civil servant' 'Unemployed'
 'Entrepreneur' 'Student' 'Paternity / maternity leave']


In [167]:
# Checking duplicates

# Check for duplicates in the DataFrame
duplicates = credit_score.duplicated()

# Count the number of duplicates
num_duplicates = duplicates.sum()

print("Number of duplicate rows:", num_duplicates)


Number of duplicate rows: 71


--The result is similar, but yours is much shorter. My code is for sake of continuity and ease of understanding as a beginner in coding. Its matter of preference, really.--

In [168]:
# Address the duplicates, if they exist

# Drop the duplicate rows
credit_score.drop_duplicates(ignore_index=True, inplace=True)

# Confirm that duplicates have been removed
duplicates_after_removal = credit_score.duplicated().sum()
print("Number of duplicate rows after removal:", duplicates_after_removal)


Number of duplicate rows after removal: 0


In [169]:
# Create a copy of the DataFrame to avoid the warning
credit_score_copy = credit_score.copy()

# Drop the duplicate rows from the copied DataFrame
credit_score_copy.drop_duplicates(ignore_index=True, inplace=True)

# Confirm that duplicates have been removed from the copied DataFrame
duplicates_after_removal = credit_score_copy.duplicated().sum()
print("Number of duplicate rows after removal:", duplicates_after_removal)

Number of duplicate rows after removal: 0


In [170]:
# Last check whether we have any duplicates
# Confirm that duplicates have been removed
duplicates_after_removal = credit_score.duplicated().sum()
print("Number of duplicate rows after removal:", duplicates_after_removal)


Number of duplicate rows after removal: 0


In [171]:
# Check the size of the dataset that you now have after your first manipulations with it
# Check the size of the original dataset
original_size = credit_score.shape

# Remove duplicates
credit_score.drop_duplicates(ignore_index=True, inplace=True)

# Check the size of the new dataset after removing duplicates
new_size = credit_score.shape

# Calculate the percentage change in dataset size
percentage_change = ((original_size[0] - new_size[0]) / original_size[0]) * 100
rounded_percentage_change = round(percentage_change, 2)

print("Size of the original dataset:", original_size)
print("Size of the new dataset after removing duplicates:", new_size)
print("Percentage change in dataset size:", rounded_percentage_change)


Size of the original dataset: (21453, 12)
Size of the new dataset after removing duplicates: (21453, 12)
Percentage change in dataset size: 0.0


In [172]:
# Check the size of the dataset that you now have after your first manipulations with it
# Check the size of the original dataset
original_size = credit_score.shape

# Create a copy of the DataFrame to avoid the warning
credit_score_copy = credit_score.copy()

# Remove duplicates from the copied DataFrame
credit_score_copy.drop_duplicates(ignore_index=True, inplace=True)

# Check the size of the new dataset after removing duplicates
new_size = credit_score_copy.shape

# Calculate the percentage change in dataset size
percentage_change = ((original_size[0] - new_size[0]) / original_size[0]) * 100
rounded_percentage_change = round(percentage_change, 2)

print("Size of the original dataset:", original_size)
print("Size of the new dataset after removing duplicates:", new_size)
print("Percentage change in dataset size:", rounded_percentage_change)

Size of the original dataset: (21453, 12)
Size of the new dataset after removing duplicates: (21453, 12)
Percentage change in dataset size: 0.0


# Working with missing values <a id='missing'></a>

In [173]:
# Find the dictionaries

# Find unique values and corresponding ID columns
education_dict = dict(zip(credit_score['education'], credit_score['education_id']))
family_status_dict = dict(zip(credit_score['family_status'], credit_score['family_status_id']))

print("Education Dictionary:")
print(education_dict)

print("Family Status Dictionary:")
print(family_status_dict)


Education Dictionary:
{"bachelor's degree": 0, 'secondary education': 1, 'some college': 2, 'primary education': 3, 'graduate degree': 4}
Family Status Dictionary:
{'Married': 0, 'Civil partnership': 1, 'Widow / widower': 2, 'Divorced': 3, 'Unmarried': 4}


### Restoring missing values in `total_income` <a id='missing-income'></a>

In [113]:
# Let's write a function that calculates the age category

def calculate_age_category(age):
    if age < 18:
        return 'Teenager'
    elif 18 <= age < 30:
        return 'Young Adult'
    elif 30 <= age < 45:
        return 'Adult'
    elif 45 <= age < 60:
        return 'Middle-aged'
    else:
        return 'Senior'

# Example usage:
age = 35
age_category = calculate_age_category(age)
print("Age Category:", age_category)


Age Category: Adult


In [178]:
# Apply the function to create a new column 'age_category'
credit_score['age_category'] = credit_score['dob_years'].apply(calculate_age_category)

# Count the number of individuals in each age category
age_category_counts = credit_score['age_category'].value_counts()

# Print the result
print('Number of individual in respective age:')
print(age_category_counts)


Number of individual in respective age:
Adult          8628
Middle-aged    7146
Young Adult    3179
Senior         2500
Name: age_category, dtype: int64


In [179]:
# Test if the function works
age = 35
age_category = calculate_age_category(age)
print("Age Category:", age_category)


Age Category: Adult


In [180]:
# Creating new column based on function
# Apply the function to create the 'age_category' column
credit_score['age_category'] = credit_score_copy['dob_years'].map(calculate_age_category)

# Display the DataFrame with the new 'age_category' column
credit_score.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,Married,0,F,Employee,0,40620.102,purchase of the house,Adult
1,1,4024.803754,36.0,secondary education,1,Married,0,F,Employee,0,17932.802,car purchase,Adult
2,0,5623.422610,33.0,secondary education,1,Married,0,M,Employee,0,23341.752,purchase of the house,Adult
3,3,4124.747207,32.0,secondary education,1,Married,0,M,Employee,0,42820.568,supplementary education,Adult
4,0,NaN,53.0,secondary education,1,Civil partnership,1,F,Retiree,0,25378.572,to have a wedding,Middle-aged


In [181]:
# Checking how values in the new column
# Check the values in the 'age_category' column
age_category_counts = credit_score['age_category'].value_counts()

# Display the counts
print(age_category_counts)


Adult          8628
Middle-aged    7146
Young Adult    3179
Senior         2500
Name: age_category, dtype: int64


In [182]:
# Create a table without missing values and print a few of its rows to make sure it looks fine
# Create a new DataFrame without missing values
complete_data = credit_score.dropna()

# Display the new DataFrame
complete_data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,Married,0,F,Employee,0,40620.102,purchase of the house,Adult
1,1,4024.803754,36.0,secondary education,1,Married,0,F,Employee,0,17932.802,car purchase,Adult
2,0,5623.422610,33.0,secondary education,1,Married,0,M,Employee,0,23341.752,purchase of the house,Adult
3,3,4124.747207,32.0,secondary education,1,Married,0,M,Employee,0,42820.568,supplementary education,Adult
5,0,926.185831,27.0,bachelor's degree,0,Civil partnership,1,M,Business,0,40922.170,purchase of the house,Young Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21447,1,2351.431934,37.0,graduate degree,4,Divorced,3,M,Employee,0,18551.846,buy commercial real estate,Adult
21448,1,4529.316663,43.0,secondary education,1,Civil partnership,1,F,Business,0,35966.698,housing transactions,Adult
21450,1,2113.346888,38.0,secondary education,1,Civil partnership,1,M,Employee,1,14347.610,property,Adult
21451,3,3112.481705,38.0,secondary education,1,Married,0,M,Employee,1,39054.888,buying my own car,Adult


In [221]:
# Look at the mean values for income based on your identified factors
# Group the data based on the identified factors and calculate the mean income for each group
income_mean_by_factors = credit_score.groupby(['education', 'family_status', 'income_type'])['total_income'].mean()

# Display the result
print(income_mean_by_factors)

education          family_status      income_type  
bachelor's degree  Civil partnership  Business         27643.468931
                                      Civil servant    27167.979245
                                      Employee         26956.959259
                                      Entrepreneur     27089.606743
                                      Retiree          26512.996748
                                                           ...     
some college       Unmarried          Employee         26875.257203
                                      Retiree          24790.570220
                   Widow / widower    Business         27089.606743
                                      Employee         27089.606743
                                      Retiree          27089.606743
Name: total_income, Length: 86, dtype: float64


In [222]:
# Look at the median values for income based on your identified factors

# Group the data based on the identified factors and calculate the median income for each group
income_median_by_factors = credit_score.groupby(['education', 'family_status', 'income_type'])['total_income'].median()

# Display the result
print(income_median_by_factors)

education          family_status      income_type  
bachelor's degree  Civil partnership  Business         27089.606743
                                      Civil servant    27089.606743
                                      Employee         27089.606743
                                      Entrepreneur     27089.606743
                                      Retiree          27089.606743
                                                           ...     
some college       Unmarried          Employee         27089.606743
                                      Retiree          27089.606743
                   Widow / widower    Business         27089.606743
                                      Employee         27089.606743
                                      Retiree          27089.606743
Name: total_income, Length: 86, dtype: float64


In [223]:
# Group the data based on multiple factors and calculate the mean and median income for each group
income_comparison = credit_score.groupby(['education', 'family_status', 'income_type', 'gender'])['total_income'].agg(['mean', 'median'])

# Display the result
print(income_comparison)

                                                                  mean  \
education         family_status     income_type   gender                 
bachelor's degree Civil partnership Business      F       27685.117160   
                                                  M       27550.256230   
                                    Civil servant F       27176.578846   
                                                  M       27134.486061   
                                    Employee      F       26957.777832   
...                                                                ...   
some college      Unmarried         Employee      M       26852.158598   
                                    Retiree       F       24790.570220   
                  Widow / widower   Business      F       27089.606743   
                                    Employee      F       27089.606743   
                                    Retiree       F       27089.606743   

                                     

In [224]:

# Step 19: Calculate mean income by income type and age category
mean_income_by_group = credit_score.groupby(['income_type', 'age_category'])['total_income'].mean()

# Step 20: Fill missing total_income values based on income type and age category
def fill_missing_income(row):
    income_type = row['income_type']
    total_income = row['total_income']
    
    if pd.isnull(total_income):
        if income_type in mean_income_by_group:
            age_category = row['age_category']
            return mean_income_by_group[income_type][age_category]
    return total_income

credit_score['total_income'] = credit_score.apply(fill_missing_income, axis=1)

# Step 21: Check the size of the dataset after manipulations
print("Size of the dataset after manipulations:", credit_score.shape)

# Step 22: Describe the new dataset
description = credit_score.describe()
display("Description of the new dataset:")
display(description)


Size of the dataset after manipulations: (21453, 15)


'Description of the new dataset:'

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000,21453.000000
mean,0.480585,2319.593235,43.474572,0.817042,0.973896,0.081154,27063.604183
std,0.756079,1994.999975,12.213068,0.548628,1.421601,0.273078,1162.346058
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,14888.651857
25%,0.000000,996.731080,33.000000,1.000000,0.000000,0.000000,27089.606743
50%,0.000000,2123.531290,43.000000,1.000000,0.000000,0.000000,27089.606743
75%,1.000000,2663.211434,53.000000,1.000000,1.000000,0.000000,27089.606743
max,5.000000,17615.563266,75.000000,4.000000,4.000000,1.000000,33421.728652


In [185]:
# Check if we got any errors
missing_values_after_fill = credit_score.isnull().sum()
print("Missing values per column after filling:")
print(missing_values_after_fill)

Missing values per column after filling:
children               0
days_employed       5549
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
age_category           0
dtype: int64


In [184]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42.0,bachelor's degree,0,Married,0,F,Employee,0,40620.102,purchase of the house,Adult
1,1,4024.803754,36.0,secondary education,1,Married,0,F,Employee,0,17932.802,car purchase,Adult
2,0,5623.422610,33.0,secondary education,1,Married,0,M,Employee,0,23341.752,purchase of the house,Adult
3,3,4124.747207,32.0,secondary education,1,Married,0,M,Employee,0,42820.568,supplementary education,Adult
4,0,NaN,53.0,secondary education,1,Civil partnership,1,F,Retiree,0,25378.572,to have a wedding,Middle-aged
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,4529.316663,43.0,secondary education,1,Civil partnership,1,F,Business,0,35966.698,housing transactions,Adult
21449,0,NaN,67.0,secondary education,1,Married,0,F,Retiree,0,24959.969,purchase of a car,Senior
21450,1,2113.346888,38.0,secondary education,1,Civil partnership,1,M,Employee,1,14347.610,property,Adult
21451,3,3112.481705,38.0,secondary education,1,Married,0,M,Employee,1,39054.888,buying my own car,Adult


In [186]:
# Checking the number of entries in the columns
# Step 1: Check the total number of values in the 'total_income' column after filling missing values
total_income_values = credit_score['total_income'].count()

# Step 2: Check the total number of values in other columns (excluding 'total_income')
other_column_values = credit_score.drop(columns=['total_income']).count().min()

# Step 3: Compare the two counts
if total_income_values == other_column_values:
    print("The total number of values in the 'total_income' column matches the number of values in other columns.")
else:
    print("There are discrepancies in the number of values between the 'total_income' column and other columns.")



There are discrepancies in the number of values between the 'total_income' column and other columns.


###  Restoring values in `days_employed`

In [183]:
# Distribution of `days_employed` medians based on your identified parameters

# Step 1: Group the data by 'income_type' and 'age_category' and calculate the median 'days_employed'
days_employed_medians = credit_score.groupby(['income_type', 'age_category'])['days_employed'].median()

# Step 2: Display the distribution of 'days_employed' medians based on the identified parameters
print("Distribution of 'days_employed' medians based on income_type and age_category:")
print(days_employed_medians)



Distribution of 'days_employed' medians based on income_type and age_category:
income_type                  age_category
Business                     Adult           1588.581162
                             Middle-aged     2044.399299
                             Senior          2470.912766
                             Young Adult      906.647054
Civil servant                Adult           2772.638532
                             Middle-aged     3668.973353
                             Senior          3318.440092
                             Young Adult     1362.645769
Employee                     Adult           1612.484985
                             Middle-aged     2124.336222
                             Senior          2669.073965
                             Young Adult     1008.784193
Entrepreneur                 Middle-aged             NaN
                             Young Adult      520.848083
Paternity / maternity leave  Adult           3296.759962
Retiree                 

In [ ]:
# Distribution of `days_employed` means based on your identified parameters

# Step 1: Group the data by 'income_type' and 'age_category' and calculate the mean 'days_employed'
days_employed_means = credit_score.groupby(['income_type', 'age_category'])['days_employed'].mean()

# Step 2: Display the distribution of 'days_employed' means based on income_type and age_category
print("Distribution of 'days_employed' means based on income_type and age_category:")
print(days_employed_means)


There were some anomalies and data issues detected during the analysis, which could impact the accuracy of mean calculations. 
The median provides a more robust estimation in such cases.

In [187]:
# Let's write a function that calculates means or medians (depending on your decision) based on your identified parameter

# Step 19: Calculate mean income by income type and age category
mean_income_by_group = credit_score.groupby(['income_type', 'age_category'])['total_income'].mean()

# Step 20: Fill missing total_income values based on income type and age category
def fill_missing_total_income(row):
    income_type = row['income_type']
    total_income = row['total_income']
    
    if pd.isnull(total_income):
        if income_type in mean_income_by_group:
            age_category = row['age_category']
            if age_category in mean_income_by_group[income_type]:
                return mean_income_by_group[income_type][age_category]
    
    # If the specific combination is missing, fill with overall median
    return mean_income_by_group.mean()

credit_score['total_income'] = credit_score.apply(fill_missing_total_income, axis=1)

# Step 21: Check the size of the dataset after manipulations
print("Size of the dataset after manipulations:", credit_score.shape)

# Step 22: Describe the new dataset
description = credit_score.describe()
print("Description of the new dataset:")
display(description)

# Step 23: Check if missing values in 'total_income' have been filled correctly
missing_values_after = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' after filling:", missing_values_after)

# Step 24: Check the distribution of 'days_employed' medians based on identified parameters
median_days_employed_by_group = credit_score.groupby(['income_type', 'age_category'])['days_employed'].median()
print("Distribution of 'days_employed' medians based on identified parameters:")
display(median_days_employed_by_group)

# Step 25: Check the distribution of 'days_employed' mean based on identified parameters
mean_days_employed_by_group = credit_score.groupby(['income_type', 'age_category'])['days_employed'].mean()
print("Distribution of 'days_employed' mean based on identified parameters:")
display(mean_days_employed_by_group)




Size of the dataset after manipulations: (21453, 13)
Description of the new dataset:


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21453.000000,15904.000000,21453.000000,21453.000000,21453.000000,21453.000000,21452.000000
mean,0.480585,2352.007285,43.474572,0.817042,0.973896,0.081154,27063.602971
std,0.756079,2300.877325,12.213068,0.548628,1.421601,0.273078,1162.373138
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,14888.651857
25%,0.000000,756.170405,33.000000,1.000000,0.000000,0.000000,27089.606743
50%,0.000000,1629.851736,43.000000,1.000000,0.000000,0.000000,27089.606743
75%,1.000000,3157.131624,53.000000,1.000000,1.000000,0.000000,27089.606743
max,5.000000,17615.563266,75.000000,4.000000,4.000000,1.000000,33421.728652


Number of missing values in 'total_income' after filling: 1
Distribution of 'days_employed' medians based on identified parameters:


income_type                  age_category
Business                     Adult           1588.581162
                             Middle-aged     2044.399299
                             Senior          2470.912766
                             Young Adult      906.647054
Civil servant                Adult           2772.638532
                             Middle-aged     3668.973353
                             Senior          3318.440092
                             Young Adult     1362.645769
Employee                     Adult           1612.484985
                             Middle-aged     2124.336222
                             Senior          2669.073965
                             Young Adult     1008.784193
Entrepreneur                 Middle-aged             NaN
                             Young Adult      520.848083
Paternity / maternity leave  Adult           3296.759962
Retiree                      Adult                   NaN
                             Middle-aged      

Distribution of 'days_employed' mean based on identified parameters:


income_type                  age_category
Business                     Adult           1986.234560
                             Middle-aged     2779.193401
                             Senior          3595.742298
                             Young Adult     1128.501894
Civil servant                Adult           3160.271436
                             Middle-aged     4544.050094
                             Senior          4343.083365
                             Young Adult     1595.684495
Employee                     Adult           2189.340880
                             Middle-aged     3056.342276
                             Senior          3850.616988
                             Young Adult     1202.858627
Entrepreneur                 Middle-aged             NaN
                             Young Adult      520.848083
Paternity / maternity leave  Adult           3296.759962
Retiree                      Adult                   NaN
                             Middle-aged      

In [189]:
def fill_missing_income(row):
    income_type = row['income_type']
    total_income = row['total_income']
    
    if pd.isnull(total_income):
        if income_type in mean_income_by_group.index.get_level_values('income_type') and \
           row['age_category'] in mean_income_by_group.index.get_level_values('age_category'):
            return mean_income_by_group[income_type][row['age_category']]
    return total_income


In [190]:
# Check that the function works
#Check the number of missing values before applying the function
missing_values_before = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' before filling:", missing_values_before)

# Apply the function to fill in missing total_income values based on income_type and age_category
credit_score['total_income'] = credit_score.apply(fill_missing_income, axis=1)

#  Check the number of missing values after applying the function
missing_values_after = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' after filling:", missing_values_after)


Number of missing values in 'total_income' before filling: 1
Number of missing values in 'total_income' after filling: 1


In [116]:
# Apply function to the income_type

print("Size of the dataset after manipulations:", credit_score.shape)

description = credit_score.describe()
print("Description of the new dataset:")
print(description)

Size of the dataset after manipulations: (21453, 12)
Description of the new dataset:
           children  days_employed     dob_years  education_id  \
count  21453.000000   15904.000000  21453.000000  21453.000000   
mean       0.480585    2352.007285     43.474572      0.817042   
std        0.756079    2300.877325     12.213068      0.548628   
min        0.000000      24.141633     19.000000      0.000000   
25%        0.000000     756.170405     33.000000      1.000000   
50%        0.000000    1629.851736     43.000000      1.000000   
75%        1.000000    3157.131624     53.000000      1.000000   
max        5.000000   17615.563266     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21453.000000  21453.000000   19350.000000  
mean           0.973896      0.081154   26787.266688  
std            1.421601      0.273078   16475.822926  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   16486

In [191]:
# Check if function worked

missing_values_before = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' before filling:", missing_values_before)

credit_score['total_income'] = credit_score.apply(fill_missing_income, axis=1)


missing_values_after = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' after filling:", missing_values_after)



Number of missing values in 'total_income' before filling: 1
Number of missing values in 'total_income' after filling: 1


In [118]:
# Find the row with the missing value in the 'total_income' column
missing_income_row = credit_score[credit_score['total_income'].isnull()]
print("Row with missing value in 'total_income' column:")
print(missing_income_row)


Row with missing value in 'total_income' column:
       children  days_employed  dob_years            education  education_id  \
12            0            NaN       65.0  secondary education             1   
26            0            NaN       41.0  secondary education             1   
29            0            NaN       63.0  secondary education             1   
41            0            NaN       50.0  secondary education             1   
55            0            NaN       54.0  secondary education             1   
...         ...            ...        ...                  ...           ...   
21417         2            NaN       47.0  secondary education             1   
21423         1            NaN       50.0  secondary education             1   
21425         0            NaN       48.0    bachelor's degree             0   
21430         1            NaN       42.0  secondary education             1   
21438         2            NaN       28.0  secondary education         

In [199]:
# Replacing missing values
# Step 1: Calculate mean income by income type and age category
mean_income_by_group = credit_score.groupby(['income_type', 'age_category'])['total_income'].mean()

# Step 2: Find the index of the maximum mean income for the corresponding income_type and age_category
max_mean_income_index = mean_income_by_group.idxmax()

# Step 3: Extract the income_type and age_category values from the index
income_type = max_mean_income_index[0]
age_category = max_mean_income_index[1]

# Step 4: Calculate the mean total_income for the corresponding income_type and age_category group
mean_total_income = mean_income_by_group[income_type][age_category]

# Step 5: Replace the missing value in the 'total_income' column with the calculated mean value
credit_score.loc[credit_score['total_income'].isnull(), 'total_income'] = mean_total_income

# Check if the missing value has been replaced
missing_values_after_fill = credit_score['total_income'].isnull().sum()
print("Number of missing values in 'total_income' after filling:", missing_values_after_fill)





Number of missing values in 'total_income' after filling: 0


In [200]:
# Create the 'age_category' column using the calculate_age_category function on the original 'credit_score' DataFrame
credit_score['age_category'] = credit_score['dob_years'].apply(calculate_age_category)

# Removes all missing data in the DataFrame
df_notna = credit_score.dropna()

# Create a groupby aggregation of categorical columns to get the median values of 'total_income' and 'days_employed'
group_edu_income = df_notna.groupby("education").agg({"total_income": "median"})
group_age_day_employed = df_notna.groupby("age_category").agg({"days_employed": "median"})

# Create a function to fill in missing values
def fill_missing_values(row, grouped_data, column, category):
    # If the data is not null, return the stored value
    if pd.notnull(row[column]):
        return row[column]
    # If the data is null/na
    else:
        # Return the mapped median category value stored in grouped data
        return grouped_data.loc[row[category]].values[0]

# Apply the function to fill in missing 'total_income' values
credit_score["total_income"] = credit_score.apply(
    fill_missing_values, grouped_data=group_edu_income,
    column="total_income", category="education", axis=1
)

# Apply the function to fill in missing 'days_employed' values
credit_score["days_employed"] = credit_score.apply(
    fill_missing_values, grouped_data=group_age_day_employed,
    column="days_employed", category="age_category", axis=1
)



In [201]:
# Check the entries in all columns - make sure we fixed all missing values

# Check the number of missing values in each column after filling
missing_values_after_fill = credit_score.isnull().sum()
print("Missing values per column after filling:")
print(missing_values_after_fill)


Missing values per column after filling:
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64


In [202]:
credit_score.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'age_category'],
      dtype='object')

## Categorization of data <a id='categorize'></a>


In [203]:
# Print the values for your selected data for categorization

print("Values in the 'family_status' column:")
print(credit_score['family_status'])

print("\nValues in the 'education' column:")
print(credit_score['education'])

print("\nValues in the 'purpose' column:")
print(credit_score['purpose'])

print("\nValues in the 'age_category' column:")
print(credit_score['age_category'])

print("\nValues in the 'children' column:")
print(credit_score['children'])

print("\nValues in the 'gender' column:")
print(credit_score['gender'])


Values in the 'family_status' column:
0                  Married
1                  Married
2                  Married
3                  Married
4        Civil partnership
               ...        
21448    Civil partnership
21449              Married
21450    Civil partnership
21451              Married
21452              Married
Name: family_status, Length: 21453, dtype: object

Values in the 'education' column:
0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21448    secondary education
21449    secondary education
21450    secondary education
21451    secondary education
21452    secondary education
Name: education, Length: 21453, dtype: object

Values in the 'purpose' column:
0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                 

In [204]:
# Check the unique values
print("\nValues in the 'family_status' column:")
print(credit_score['family_status'].unique())

print("\nValues in the 'education' column:")
print(credit_score['education'].unique())

print("\nValues in the 'purpose' column:")
print(credit_score['purpose'].unique())

print("\nValues in the 'age_category' column:")
print(credit_score['age_category'].unique())

print("\nValues in the 'children' column:")
print(credit_score['children'].unique())

print("\nValues in the 'gender' column:")
print(credit_score['gender'].unique())


Values in the 'family_status' column:
['Married' 'Civil partnership' 'Widow / widower' 'Divorced' 'Unmarried']

Values in the 'education' column:
["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']

Values in the 'purpose' column:
['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car

In [205]:
# Let's write a function to categorize the data based on common topics
def categorize_data(column, categories_mapping):
    return column.map(categories_mapping)


In [ ]:
#Create a column with the categories and count the values for them
# Function for categorizing the data based on purpose
def categorize_purpose(text):
    if {"property", "estate", "house", "housing"}.intersection(text.lower().split(" ")):
        return "Real estate"
    elif {"car"}.intersection(text.lower().split(" ")):
        return "Car"
    elif {"education"}.intersection(text.lower().split(" ")):
        return "Education"
    elif {"wedding"}.intersection(text.lower().split(" ")):
        return "Wedding"
    else:
        return "Other"

# Apply the function to create the 'purpose_category' column
credit_score['purpose_category'] = credit_score['purpose'].apply(categorize_purpose)

# Function for categorizing the data based on family status
def categorize_family_status(family_status):
    if family_status.lower() in ["married", "civil partnership"]:
        return "Married"
    elif family_status.lower() == "widow / widower":
        return "Widow/Widower"
    elif family_status.lower() == "divorced":
        return "Divorced"
    elif family_status.lower() in ["unmarried", "single / not married"]:
        return "Single"
    else:
        return "Other"

# Apply the function to create the 'family_status_category' column
credit_score['family_status_category'] = credit_score['family_status'].apply(categorize_family_status)

# Print the unique values in the new columns
print("Values in the 'purpose_category' column:")
print(credit_score['purpose_category'].unique())

print("\nValues in the 'family_status_category' column:")
print(credit_score['family_status_category'].unique())

In [206]:
credit_score.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'age_category'],
      dtype='object')

In [207]:
credit_score.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [208]:
# Looking through all the numerical data in your selected column for categorization
age_bins = [0, 30, 45, 60, 100]
age_labels = ['Young', 'Middle-aged', 'Senior', 'Elderly']

credit_score['age_category'] = pd.cut(credit_score['dob_years'], bins=age_bins, labels=age_labels)

print("Values in the 'age_category' column:")
print(credit_score['age_category'].unique())

Values in the 'age_category' column:
['Middle-aged', 'Senior', 'Young', 'Elderly']
Categories (4, object): ['Young' < 'Middle-aged' < 'Senior' < 'Elderly']


In [209]:
# Getting summary statistics for the column

summary_stats = credit_score['dob_years'].describe()

print("Summary statistics for the 'dob_years' column:")
print(summary_stats)

Summary statistics for the 'dob_years' column:
count    21453.000000
mean        43.474572
std         12.213068
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64


In [210]:
# Creating function for categorizing into different numerical groups based on ranges

def categorize_numerical(value, ranges):
    for i, r in enumerate(ranges):
        if value <= r:
            return f'Group {i + 1}'
    return f'Group {len(ranges) + 1}'

In [211]:
# Creating column with categories
# Example ranges for age column
age_ranges = [30, 45, 60]

# Create a new column 'age_category' based on the 'dob_years' column
credit_score['age_category'] = credit_score['dob_years'].apply(lambda age: categorize_numerical(age, age_ranges))

In [212]:
# Count each categories values to see the distribution

# Count the values in the 'age_category' column
age_category_distribution = credit_score['age_category'].value_counts()

# Print the distribution
print("Distribution of values in the 'age_category' column:")
print(age_category_distribution)

Distribution of values in the 'age_category' column:
Group 2    8587
Group 3    7024
Group 1    3716
Group 4    2126
Name: age_category, dtype: int64


In [213]:
# Define a dictionary to map the current age_category values to the desired strings
age_category_mapping = {
    'Group 1': 'Teenager',
    'Group 2': 'Young Adult',
    'Group 3': 'Adult',
    'Group 4': 'Middle-aged',
    'Group 5': 'Senior'
}

# Use the map method to replace the values in the 'age_category' column
credit_score['age_category'] = credit_score['age_category'].map(age_category_mapping)

# Count the values in the 'age_category' column again to see the updated distribution
age_category_distribution = credit_score['age_category'].value_counts()

# Print the updated distribution
print("Distribution of values in the 'age_category' column:")
print(age_category_distribution)


Distribution of values in the 'age_category' column:
Young Adult    8587
Adult          7024
Teenager       3716
Middle-aged    2126
Name: age_category, dtype: int64


## Checking the Hypotheses <a id='hypo'></a>


**Is there a correlation between having children and paying back on time?**

In [214]:
# Check the children data and paying back on time


# Calculating default-rate based on the number of children

# Group the data by 'children' and calculate the mean of 'debt' for each group
default_rate_by_children = credit_score.groupby('children')['debt'].mean() * 100

# Print the default rate for each number of children
print("Default rate based on the number of children:")
print(default_rate_by_children)

Default rate based on the number of children:
children
0    7.544358
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64


**Conclusion**

It appears that clients with more children tend to have slightly higher default rates compared to those with fewer or no children.

**Is there a correlation between family status and paying back on time?**

In [215]:
# Check the family status data and paying back on time
# Calculating default-rate based on family status

# Group the data by 'family_status' and calculate the mean of 'debt' for each group
default_rate_by_family_status = credit_score.groupby('family_status')['debt'].mean() * 100

# Print the default rate for each family status in percentage format
print("Default rate based on family status:")
print(default_rate_by_family_status)

Default rate based on family status:
family_status
Civil partnership    9.349398
Divorced             7.112971
Married              7.545182
Unmarried            9.750890
Widow / widower      6.569343
Name: debt, dtype: float64


**Conclusion**

It appears that clients who are unmarried or in a civil partnership have a higher default rate, in turn, higher credit risk

**Is there a correlation between income level and paying back on time?**

In [216]:
# Check the income level data and paying back on time
# Calculating default-rate based on income level

# Step 1: Categorize the total_income into income levels or groups
income_bins = [0, 20000, 40000, 60000, 80000, 100000, float('inf')]
income_labels = ['<20k', '20k-40k', '40k-60k', '60k-80k', '80k-100k', '100k+']
credit_score['income_level'] = pd.cut(credit_score['total_income'], bins=income_bins, labels=income_labels)

# Step 2: Group the data by the income level and calculate the mean of 'debt' for each group
default_rate_by_income_level = credit_score.groupby('income_level')['debt'].mean() * 100

# Step 3: Set missing default rate values to 0
default_rate_by_income_level = default_rate_by_income_level.fillna(0)

# Step 4: Print the default rate for each income level
print("Default rate based on income level:")
print(default_rate_by_income_level)


Default rate based on income level:
income_level
<20k        0.000000
20k-40k     8.115793
40k-60k     0.000000
60k-80k     0.000000
80k-100k    0.000000
100k+       0.000000
Name: debt, dtype: float64


**Conclusion**

Higher-income individuals tend to have lower default rates since they have more resources to manage their debts. Conversely, lower-income individuals may have higher default rates as they may struggle to make timely payments due to financial constraints.

**How does credit purpose affect the default rate?**

In [219]:
def categorize_credit_purpose(purpose):
    if 'wedding' in purpose:
        return 'Wedding'
    elif 'car' in purpose:
        return 'Car'
    elif 'education' in purpose or 'university' in purpose:
        return 'Education'
    elif 'real estate' in purpose or 'property' in purpose or 'house' in purpose:
        return 'Real Estate'
    else:
        return 'Other'

In [220]:
# Check the percentages for default rate for each credit purpose and analyze them
# Apply the function to create the 'purpose_category' column
credit_score['purpose_category'] = credit_score['purpose'].apply(categorize_credit_purpose)

# Check the percentages for default rate for each credit purpose and analyze them
# Step 1: Group the data by the purpose_category and calculate the mean of 'debt' for each group
default_rate_by_purpose = credit_score.groupby('purpose_category')['debt'].mean() * 100

# Step 2: Print the default rate for each credit purpose
print("Default rate based on credit purpose:")
print(default_rate_by_purpose)


Default rate based on credit purpose:
purpose_category
Car            9.359034
Education      9.181692
Other          7.263294
Real Estate    7.332959
Wedding        8.003442
Name: debt, dtype: float64


**Conclusion**

According to the dafault rate, most of them are prefered to have a car and supplementary education, as to ease of mobility and upskill oneself for better prespect and future.

# General Conclusion <a id='end'></a>


<b>Conclusions:<b>

1. Missing Values: The dataset contained missing values in the 'days_employed' and 'total_income' columns. 
>To address this, we calculated the median income for each combination of income type and age category and filled in the missing 'total_income' values accordingly. 
>However, we observed one remaining missing value in the 'total_income' column, which might have occurred due to an insufficient number of entries in a specific income type and age category.

1. Duplicates: The dataset had some duplicate rows, which we identified and removed to ensure data integrity and accurate analysis.

1. Incorrect Data: We encountered problematic data in the 'children' and 'dob_years' columns. 
>Some rows had negative or excessively high numbers of children, and some had zero values for age. 
#We addressed these issues by replacing negative and excessively high values with appropriate ones and filling in the zero values for age with the median age.

1. Categorization: We categorized data in the 'family_status' and 'purpose' columns to analyze default rates based on different factors. 
>The default rates varied among different family statuses and credit purposes, indicating potential correlations between these factors and the likelihood of repayment.

1. Default Rates: We calculated default rates based on the number of children, family status, income level, and credit purpose. 
>The default rates showed variations among different groups, providing insights into the relationships between these factors and loan repayment behavior.

1. Income Level Issue: We noticed that the default rate for income levels between 40k-60k and 60k-80k was 0%. 
>This could indicate possible data issues or errors in the income level categorization. 
>It is essential for data validation and double-check the income level categorization to ensure accuracy in the results.

1. Limitations: The analysis conducted here provides valuable insights, but it has some limitations. 
>The dataset may not be entirely representative of the entire population, and the findings should be interpreted with caution. 
>Additionally, the default rates should be analyzed in conjunction with other factors, such as loan terms, credit history, and loan amounts, to obtain a more comprehensive understanding of default risk.

1. Further Analysis: To gain deeper insights, further analysis and modeling could be performed, such as logistic regression, to identify the most influential factors affecting loan default.



In conclusion, our data preprocessing and analysis revealed valuable information about the dataset, default rates, and factors influencing loan repayment. Addressing missing values, duplicates, and incorrect data allowed for more accurate analysis and interpretation of the findings. However, additional data verification and validation would be required to ensure the accuracy and reliability of the results.